# Data Preparation for YOLO Instance Segmentation

This notebook prepares the food-recognition-2022 dataset for training a YOLOv11n-seg model. We will:
1. Load and parse the annotations from the JSON files
2. Create the YOLO directory structure
3. Convert instance segmentation masks to YOLO format
4. Create a dataset.yaml file for training

In [ ]:
# Import required libraries
import os
import json
import shutil
from pathlib import Path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

## Setup Paths

In [ ]:
# Define dataset paths
DATA_ROOT = os.path.join("..", "datasets", "food-recognition-2022")
TRAIN_DIR = os.path.join(DATA_ROOT, "training")
VAL_DIR = os.path.join(DATA_ROOT, "val")
TEST_DIR = os.path.join(DATA_ROOT, "test", "images")

# Create YOLO dataset directory
YOLO_DATASET_DIR = os.path.join("..", "datasets", "yolo_food_dataset")
os.makedirs(YOLO_DATASET_DIR, exist_ok=True)

# Define YOLO directory structure
for split in ["train", "val"]:
    os.makedirs(os.path.join(YOLO_DATASET_DIR, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(YOLO_DATASET_DIR, split, "labels"), exist_ok=True)

# Verify paths
print(f"Training directory: {TRAIN_DIR}")
print(f"Validation directory: {VAL_DIR}")
print(f"YOLO dataset directory: {YOLO_DATASET_DIR}")

## Load Annotations

In [ ]:
# Load annotations from JSON files
def load_annotations(json_path):
    with open(json_path, 'r') as f:
        return json.load(f)

# Load train and validation annotations
train_annotations = load_annotations(os.path.join(TRAIN_DIR, "annotations.json"))
val_annotations = load_annotations(os.path.join(VAL_DIR, "annotations.json"))

# Extract class information and create a class mapping
categories = {cat['id']: cat['name'] for cat in train_annotations['categories']}
class_ids = {cat_id: idx for idx, cat_id in enumerate(categories.keys())}

print(f"Found {len(categories)} categories:")
for cat_id, name in categories.items():
    print(f"  {class_ids[cat_id]}: {name} (original id: {cat_id})")

print(f"\nTraining images: {len(train_annotations['images'])}")
print(f"Validation images: {len(val_annotations['images'])}")
print(f"Training annotations: {len(train_annotations['annotations'])}")
print(f"Validation annotations: {len(val_annotations['annotations'])}")

## Process Annotations

Convert the COCO format annotations to YOLO format:
- YOLO expects one text file per image in the "labels" directory
- Each line in the text file represents one object: `class_id x_center y_center width height x1 y1 x2 y2 ... xn yn`
- The first 5 values are the class ID and normalized bounding box coordinates (center_x, center_y, width, height)
- The remaining values are normalized polygon points of the segmentation mask (x1, y1, x2, y2, ...)

In [ ]:
def process_dataset(annotations, src_dir, dest_images_dir, dest_labels_dir):
    # Create image_id to filename mapping
    image_dict = {img['id']: img for img in annotations['images']}
    
    # Group annotations by image_id
    image_annotations = {}
    for ann in annotations['annotations']:
        image_id = ann['image_id']
        if image_id not in image_annotations:
            image_annotations[image_id] = []
        image_annotations[image_id].append(ann)
    
    # Process each image
    for image_id, anns in tqdm(image_annotations.items(), desc="Processing images"):
        image_info = image_dict[image_id]
        filename = image_info['file_name']
        
        # Get image dimensions
        img_width = image_info['width']
        img_height = image_info['height']
        
        # Copy image to destination
        src_path = os.path.join(src_dir, filename)
        dest_path = os.path.join(dest_images_dir, filename)
        shutil.copy(src_path, dest_path)
        
        # Create YOLO annotation file
        label_lines = []
        
        for ann in anns:
            category_id = ann['category_id']
            yolo_class_id = class_ids[category_id]
            
            # Convert bbox to YOLO format (normalized x_center, y_center, width, height)
            x, y, w, h = ann['bbox']
            x_center = (x + w/2) / img_width
            y_center = (y + h/2) / img_height
            width = w / img_width
            height = h / img_height
            
            # Convert segmentation polygon to YOLO format (normalized coordinates)
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                polygon = ann['segmentation'][0]  # Take the first polygon if there are multiple
                points = []
                for i in range(0, len(polygon), 2):
                    x_coord = polygon[i] / img_width
                    y_coord = polygon[i+1] / img_height
                    points.extend([x_coord, y_coord])
                
                # Create YOLO format line
                line = f"{yolo_class_id} {x_center} {y_center} {width} {height}"
                for p in points:
                    line += f" {p}"
                label_lines.append(line)
        
        # Write label file
        if label_lines:
            label_filename = os.path.splitext(filename)[0] + '.txt'
            with open(os.path.join(dest_labels_dir, label_filename), 'w') as f:
                f.write('\n'.join(label_lines))

# Process training and validation datasets
print("Processing training dataset...")
process_dataset(train_annotations, os.path.join(TRAIN_DIR, "images"), 
                os.path.join(YOLO_DATASET_DIR, "train", "images"), 
                os.path.join(YOLO_DATASET_DIR, "train", "labels"))

print("Processing validation dataset...")
process_dataset(val_annotations, os.path.join(VAL_DIR, "images"), 
                os.path.join(YOLO_DATASET_DIR, "val", "images"), 
                os.path.join(YOLO_DATASET_DIR, "val", "labels"))

## Create Dataset Configuration

In [ ]:
# Create dataset.yaml for YOLO training
yaml_content = f"""# YOLO Food Recognition Dataset
path: {YOLO_DATASET_DIR}
train: train/images
val: val/images

# Classes
names:
"""

for cat_id, name in sorted([(class_ids[k], v) for k, v in categories.items()]):
    yaml_content += f"  {cat_id}: '{name}'\n"

# Write YAML file
yaml_path = os.path.join(YOLO_DATASET_DIR, "dataset.yaml")
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"Dataset configuration saved to {yaml_path}")
print("Sample of the YAML file:")
print("\n".join(yaml_content.split("\n")[:10]) + "\n...")

## Verify Dataset

In [ ]:
# Count files in dataset directories
train_images = len(list(Path(os.path.join(YOLO_DATASET_DIR, "train", "images")).glob("*")))
train_labels = len(list(Path(os.path.join(YOLO_DATASET_DIR, "train", "labels")).glob("*.txt")))
val_images = len(list(Path(os.path.join(YOLO_DATASET_DIR, "val", "images")).glob("*")))
val_labels = len(list(Path(os.path.join(YOLO_DATASET_DIR, "val", "labels")).glob("*.txt")))

print(f"Train images: {train_images}")
print(f"Train labels: {train_labels}")
print(f"Validation images: {val_images}")
print(f"Validation labels: {val_labels}")

# Sample a few labels to verify
def sample_label(labels_dir, n=3):
    label_files = list(Path(labels_dir).glob("*.txt"))
    if not label_files:
        return "No label files found"
    
    samples = []
    for i in range(min(n, len(label_files))):
        with open(label_files[i], 'r') as f:
            content = f.read().strip()
            samples.append(f"File: {label_files[i].name}\n{content[:300]}{'...' if len(content) > 300 else ''}")
    return "\n\n".join(samples)

print("\nSample training labels:")
print(sample_label(os.path.join(YOLO_DATASET_DIR, "train", "labels")))

## Visualize Sample Annotations

Let's visualize some images with their annotations to verify the conversion was successful.

In [ ]:
def draw_yolo_annotations(image_path, label_path):
    # Load image
    img = Image.open(image_path)
    img_width, img_height = img.size
    
    # Create figure
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(img)
    
    # Load labels
    if not os.path.exists(label_path):
        ax.set_title(f"Image: {os.path.basename(image_path)}\nNo annotations found")
        return fig
    
    with open(label_path, 'r') as f:
        lines = f.readlines()
    
    # Draw each annotation
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        class_name = next((v for k, v in categories.items() if class_ids[k] == class_id), f"Class {class_id}")
        
        # Bounding box (normalized xywh -> pixel xyxy)
        x_center, y_center, width, height = map(float, parts[1:5])
        x_min = (x_center - width/2) * img_width
        y_min = (y_center - height/2) * img_height
        box_width = width * img_width
        box_height = height * img_height
        
        # Draw rectangle
        rect = plt.Rectangle((x_min, y_min), box_width, box_height,
                            fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        ax.text(x_min, y_min, class_name, color='white',
                bbox=dict(facecolor='red', alpha=0.5))
        
        # Draw segmentation polygon if available
        if len(parts) > 5:
            poly_points = parts[5:]
            polygon = []
            for i in range(0, len(poly_points), 2):
                x = float(poly_points[i]) * img_width
                y = float(poly_points[i+1]) * img_height
                polygon.append((x, y))
            
            poly_xs, poly_ys = zip(*polygon)
            ax.plot(poly_xs, poly_ys, '-', color='lime', linewidth=2)
    
    ax.set_title(f"Image: {os.path.basename(image_path)}")
    ax.axis('off')
    return fig

# Visualize a few random examples
def visualize_random_samples(split="train", n=3):
    images_dir = os.path.join(YOLO_DATASET_DIR, split, "images")
    labels_dir = os.path.join(YOLO_DATASET_DIR, split, "labels")
    
    image_files = list(Path(images_dir).glob("*"))
    if not image_files:
        print(f"No images found in {images_dir}")
        return
    
    # Sample n random images
    import random
    sample_images = random.sample(image_files, min(n, len(image_files)))
    
    for img_path in sample_images:
        base_name = os.path.splitext(img_path.name)[0]
        label_path = os.path.join(labels_dir, f"{base_name}.txt")
        fig = draw_yolo_annotations(img_path, label_path)
        plt.tight_layout()
        plt.show()

print("Visualizing training samples:")
visualize_random_samples("train", 3)

print("Visualizing validation samples:")
visualize_random_samples("val", 3)

## Conclusion

This notebook has:
1. Converted the food recognition dataset from COCO format to YOLO format
2. Created the proper directory structure for training
3. Generated a dataset.yaml file with class information
4. Verified the conversion with visualizations

Your dataset is now ready for training with YOLOv11n-seg. You can use the training code provided above to start training your model.